In [1]:
#
import sys
import os
sys.path.append("/home/sergey/drclinics/common")
sys.path.append("/home/sergey/drclinics/reports")
#


import pandas as pd
from report_utils import query_by_name, send_mail
from configs import load_json_config

from connect_platform import connect_telemed
from log import log
import datetime
from utils import get_path, DATETIME_FORMAT

from universal_connection import UniversalConnection, DBType

from log import log

In [3]:
phones = pd.read_excel('Webmed-pats-no-cons3_202006051325_unsubscribe.xlsx')

In [11]:
phone_list = [str(x) for x in(phones.patient_phone.unique())]

In [17]:
# phone_list

In [18]:
phone_string = "'" + "', '".join(phone_list) + "'"

In [20]:
# phone_string

In [21]:
sql = f"""
select 	a.id app_id,
		date(a.finished at time zone 'UTC') cons_date,
		concat_ws(' ', p_patient.last_name, p_patient.first_name, p_patient.middle_name) patient_fio,
		p_patient.sex, 
		p_patient.birthday patient_birthday,
		p_patient.phone patient_phone,
		p_patient.email,
		case
  		when "source"='SCHEDULED' and specialty_id is not null then (
			select trim(name)
			from specialty s
			where s.id = a.specialty_id
		)
	    else (
			select trim(s.name)
			from specialty s
			inner join doctor_specialty ds
			   on ds.specialty_id = s.id
			inner join doctor_specialty_treatment dst
			   on dst.doctor_specialty_id = ds.id
			where dst.treatment_id = a.treatment_id
			limit 1 -- из-за одной криво заведенной записи.
		) end as specialty
from appointment a
left join patient on a.patient_id = patient.id 
left join person p_patient on patient.person_id = p_patient.id
--
where a.finished at time zone 'UTC' > '2020-06-05'
and a.good 
and not (
      	a.patient_id in (
	      	select patient_id
	      	from patient_categories pcat
	      	inner join reference rf
	      	   on rf.id=pcat.reference_id
	      	where rf.code='TEST'
	      	)
	    or lower(a.report_comment)='тест'
      	)
--
and
p_patient.phone in ({phone_string})

"""

In [22]:
connection = UniversalConnection('../../../.credentials/telemed/prom.cfg', DBType.Postgres)
data = connection.query(sql)
connection.close()

2020-06-11 13:49:32 connect to postgres database using config file "../../../.credentials/telemed/prom.cfg"
2020-06-11 13:49:32 creating ssh tunnel to 172.16.100.19 as root...
2020-06-11 13:49:32 connect postgres using parameters:
                    database: telemed
				    user: norekhov
				    password: ***masked***
				    host: localhost
				    port: 43327
2020-06-11 13:49:32 @telemed: execute sql:
				    SET TIME ZONE 'Europe/Moscow'
				    None
2020-06-11 13:49:32 @telemed query:
                    select 	a.id app_id,
				    		date(a.finished at time zone 'UTC') cons_date,
				    		concat_ws(' ', p_patient.last_name, p_patient.first_name, p_patient.middle_name) patient_fio,
				    		p_patient.sex, 
				    		p_patient.birthday patient_birthday,
				    		p_patient.phone patient_phone,
				    		p_patient.email,
				    		case
				      		when "source"='SCHEDULED' and specialty_id is not null then (
				    			select trim(name)
				    			from specialty s
				    			whe

In [24]:
data.to_excel('cons-asfter-sms.xlsx', index=False)